In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import BlockRNNModel
from darts.metrics import smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity',
    'Soil_Moisture', 'Soil_Temperature'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 168, step=12)
    hidden_size = trial.suggest_int("hidden_size", 32, 256, step=8)
    layer_size = trial.suggest_int("layer_size", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.05)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=8)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        opt_kwargs = {"lr": lr, "weight_decay": weight_decay}

        model = BlockRNNModel(
            model="LSTM",
            model_name=f"model_{fold+1}",
            input_chunk_length=input_chunk_length,
            output_chunk_length=24,
            hidden_dim=hidden_size,
            n_rnn_layers=layer_size,
            n_epochs=30,
            batch_size=batch_size,
            dropout=dropout,
            add_encoders=encoders,
            pl_trainer_kwargs=pl_trainer_kwargs,
            loss_fn=torch.nn.L1Loss(),
            optimizer_cls=torch.optim.Adam,
            lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 7, "min_lr": 1e-7},
            save_checkpoints=True,
            force_reset=True,
            random_state=42,
            optimizer_kwargs=opt_kwargs,
        )
        model.fit(
            series=train_scaled,
            val_series=val_scaled,
            verbose=False,
            dataloader_kwargs={"num_workers": 11},
        )

        try:
            loaded_model = model.load_from_checkpoint(f"model_{fold+1}", best=True)
            print(f"Model loaded from checkpoint for trial {trial.number}, fold {fold}")
        except FileNotFoundError:
            print(f"Checkpoint not found. Using the in-memory trained model.")
            loaded_model = model

        forecasts = loaded_model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )

        overall_smape_val = smape(val_scaled, forecasts)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "fold_smape": overall_smape_list,
        "mean_smape": mean_smape,
        "hyperparameters": trial.params
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_smape if not np.isnan(mean_smape) else float("inf")

In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 7
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-06-14 09:24:11,918] A new study created in memory with name: no-name-447044e7-aa2a-452e-9aa4-98f095e71574
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 1/3


Metric val_loss improved. New best score: 12.468
Metric val_loss improved by 0.014 >= min_delta = 0.0005. New best score: 12.454
Metric val_loss improved by 0.040 >= min_delta = 0.0005. New best score: 12.414
Metric val_loss improved by 0.092 >= min_delta = 0.0005. New best score: 12.322
Metric val_loss improved by 0.035 >= min_delta = 0.0005. New best score: 12.287
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 12.285
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.285
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.283
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.282
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.281
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score: 12.279
Metric val_loss improved by 0.001 >= min_delta = 0.0005. New best score: 12.278
Metric val_loss improved by 0.002 >= min_delta = 0.0005. New best score

Model loaded from checkpoint for trial 0, fold 0


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Current value: 12.255443499343347, Current params: {'input_chunk_length': 48, 'hidden_size': 256, 'layer_size': 2, 'dropout': 0.45000000000000007, 'batch_size': 112, 'lr': 0.0023709680689192594, 'weight_decay': 3.9992979401092636e-05}
Best value: 60.23985809151379, Best params: {'input_chunk_length': 24, 'hidden_size': 256, 'layer_size': 2, 'dropout': 0.5, 'batch_size': 112, 'lr': 0.0023152837587172353, 'weight_decay': 0.0001372149130909975}

Fold 1/3


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2025-06-14 11:15:21,024] Trial 47 pruned. Trial was pruned at epoch 0.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 12.373206141678343, Current params: {'input_chunk_length': 72, 'hidden_size': 248, 'layer_size': 2, 'dropout': 0.5, 'batch_size': 96, 'lr': 0.0006524530664997378, 'weight_decay': 4.294417629432931e-05}
Best value: 60.23985809151379, Best params: {'input_chunk_length': 24, 'hidden_size': 256, 'layer_size': 2, 'dropout': 0.5, 'batch_size': 112, 'lr': 0.0023152837587172353, 'weight_decay': 0.0001372149130909975}

Fold 1/3


Metric val_loss improved. New best score: 12.261
[I 2025-06-14 11:15:26,191] Trial 48 pruned. Trial was pruned at epoch 1.


Current value: 12.220623501869895, Current params: {'input_chunk_length': 36, 'hidden_size': 80, 'layer_size': 1, 'dropout': 0.4, 'batch_size': 112, 'lr': 0.003007219260903446, 'weight_decay': 0.00013612903706714522}
Best value: 60.23985809151379, Best params: {'input_chunk_length': 24, 'hidden_size': 256, 'layer_size': 2, 'dropout': 0.5, 'batch_size': 112, 'lr': 0.0023152837587172353, 'weight_decay': 0.0001372149130909975}


In [8]:
print(f"Best MASE Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best MASE Value (Minimum): 60.23985809151379
Best Parameters: {'input_chunk_length': 24, 'hidden_size': 256, 'layer_size': 2, 'dropout': 0.5, 'batch_size': 112, 'lr': 0.0023152837587172353, 'weight_decay': 0.0001372149130909975}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
